### SQL Interview Questions

using MySQL and Python

Installing mysql on mac is easy.

 - https://dev.mysql.com/doc/refman/5.7/en/macos-installation-pkg.html
 - https://dev.mysql.com/downloads/mysql/

You download Intel-based dmg installer (mysql-8.0.28-macos11-x86_64.dmg)
and follow instructions.

Note:
  About new Mac with M1 chip and python.
  As of this writing in January 2022 I recommend to use Intel-based installers.
  Native (to ARM M1 chip) version of python has appeared in July of 2021.
  But it is still too new, many modules are not supported yet. 
  Even today if you look at the official anaconda download page,
  there is no support for ARM.

Note:
<br>You will be asked to create root password during the installation.
<br>Or you can do it from terminal: 
``` code
mysqladmin -u root password YOURNEWPASSWORD
```

Add the root password into config file in your home directory:

```
vi ~/.my.cnf

[mysql]
user=root
password=YOURNEWPASSWORD
```

save - and change permissions:
```
chmod 0600 .my.cnf
```

edit .bashrc - add path to mysql executables like this:

```
PATH=$PATH:/usr/local/mysql/bin
```

Restart terminal.
Now you can start mysql prompt with "mysql" command

Once on mysql prompt, you can start working with SQL:

```
show databases;
create database testdb;
use testdb;
create table t1 ( i1 int null, c1 varchar(80) null );
show tables;
insert into t1 values (1,'a');
insert into t1 values (2,'b');
select * from t1;
exit
```

To work with mysql from python - look at this python file:

https://github.com/lselector/setup_computer/blob/master/py_lib/myutil_mysql.py

```
pip install mysql-connector-python
```

In [1]:
import os, sys

# add current directory to search path
if "." not in sys.path:
    sys.path = ["."] + sys.path

for p in sys.path:
    print("    ", p)

     .
     /Users/maltethesenvitz/Documents/GitHub/db
     /Users/maltethesenvitz/Documents/GitHub/db
     /Users/maltethesenvitz/docs/py_lib
     /Users/maltethesenvitz/anaconda3/lib/python39.zip
     /Users/maltethesenvitz/anaconda3/lib/python3.9
     /Users/maltethesenvitz/anaconda3/lib/python3.9/lib-dynload
     
     /Users/maltethesenvitz/anaconda3/lib/python3.9/site-packages
     /Users/maltethesenvitz/anaconda3/lib/python3.9/site-packages/aeosa
     /Users/maltethesenvitz/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg
     /Users/maltethesenvitz/anaconda3/lib/python3.9/site-packages/IPython/extensions
     /Users/maltethesenvitz/.ipython


In [2]:
import mysql.connector as connection
import pandas as pd

mysql_pwd = os.getenv('MYSQL_PWD')
mysql_conn = None # connection
try:
    cnx = connection.connect(
        host     = "localhost", 
        database = 'testdb',
        user     = "root", 
        passwd   = mysql_pwd,
        use_pure = True)

    sql = "select * from t1;"
    
    df = pd.read_sql(sql, cnx)

    print(df)

except Exception as e:
    cnx.close()
    print(str(e))

   i1     c1
0   1      a
1   2      b
2   2  IpTcH
3   7  OqzKu
4  10  eJQbY


In [3]:
print(cnx)

In [4]:
import myutil_mysql
from myutil_mysql import *
dir(myutil_mysql)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'connect_to_mysql',
 'dbquote',
 'do_query',
 'do_sql',
 'dt',
 'mysql',
 'np',
 'os',
 'pd',
 're',
 'remove_extra_indents',
 'sys',
 'time']

In [5]:
# functions in myutil_mysql
# remove_extra_indents(sql) 
#    - returns SQL string without extra indents on the left
# dbquote(form, val)
#    - convenience function to format your value 
#      "val" into a string with quotes
# connect_to_mysql(myuser=None, mypasswd) 
#    - connects to the database
#    - returns tuple (cnx, 0) 
# do_query(cnx, sql) - runs sql query, 
#    - returns tuple (df, error_code)
# do_sql(cnx, sql) - runs sql, returns error code

In [6]:
# create connection
cnx,err = connect_to_mysql(myuser="root", mypasswd=os.getenv("MYSQL_PWD")) 

In [7]:
df,err = do_query(cnx, "select * from t1;")
print(df)
if err != 0:
    print("ERROR:",err)

   i1     c1
0   1      a
1   2      b
2   2  IpTcH
3   7  OqzKu
4  10  eJQbY


In [8]:
# example using do_query() function
df,err = do_query(cnx, "show tables;")
print(df)
if err != 0:
    print("ERROR:",err)

  Tables_in_testdb
0               t1
1               t2


In [9]:
sql = """
create table t2 (
  a int null,
  b int null,
  c varchar(40) null
);
"""
err = do_sql(cnx, sql)
if err != 0:
    print("ERROR:",err)

1050 (42S01): Table 't2' already exists
ERROR: 1050 (42S01): Table 't2' already exists


In [10]:
# function to generate random string
import string
from random import choices, randint

In [11]:
def rstr(mylen=5):
    return ''.join(choices(string.ascii_letters,k=mylen))

In [12]:
# example using do_sql() function
ri = randint(0,10)
rs = rstr() # random string
err = do_sql(cnx, f"insert into t1 values({ri},'{rs}');")
if err != 0:
    print("ERROR:",err)

In [13]:
# example using do_query() function
df,err = do_query(cnx, "select * from t1 where c1 != 'cc';")
print(df)
if err != 0:
    print("ERROR:",err)

   i1     c1
0   1      a
1   2      b
2   2  IpTcH
3   7  OqzKu
4  10  eJQbY
5   9  ttcnW


In [14]:
# Question
# Given table with two columns: name and salary
# Make a query to return name of someone who has 3rd largest salary
# Note: if largest salaries are 100,100,100,100,90,90,90,90,80,80,80,80,...
# it should return someone with salary = 80

# todo:
#   if table "emp_salary" does not exist:
#      create it and populate with data
#  run query to return name and salary for the 3rd largest salary value

pass

sql = """
show tables where Tables_in_testdb='emp_salary';
"""
df,err = do_query(cnx, sql)
if len(df) >=1:
    print("exists")
else:
    print("not exists, need to create table and insert data")
    pass # create and insert



not exists, need to create table and insert data


##  SQL Interview Question

### Setup + Creating Tables

In [15]:
# Check if Database ORG has been created

import mysql.connector as connection
import pandas as pd

mysql_pwd = os.getenv('MYSQL_PWD')
mysql_conn = None # connection
try:
    cnx = connection.connect(
        host     = "localhost", 
        database = 'ORG',
        user     = "root", 
        passwd   = mysql_pwd,
        use_pure = True)

    sql = "show databases;"
    
    df = pd.read_sql(sql, cnx)

    
    print(df)

except Exception as e:
    cnx.close()
    print(str(e))

             Database
0  information_schema
1          mydatabase
2               mysql
3                 ORG
4  performance_schema
5                 sys
6              testdb


In [20]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=mysql_pwd,
  database="ORG"
)

cursor = mydb.cursor()
cursor.execute("SHOW TABLES")

for table_name in cursor:
   print(table_name)

In [22]:
# drop table worker
# mydb = mysql.connector.connect(
#   host="localhost",
#   user="root",
#   password=mysql_pwd,
#   database="ORG"
# )

# mycursor = mydb.cursor()

# sql = "DROP TABLE Worker;"

# mycursor.execute(sql)

In [23]:
# create table worker

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=mysql_pwd,
  database="ORG"
)

mycursor = mydb.cursor()

mycursor.execute("""CREATE TABLE Worker 
                 (WORKER_ID INT PRIMARY KEY AUTO_INCREMENT,
                 FIRST_NAME CHAR(25),
                 LAST_NAME CHAR(25),
                 SALARY INT(15),
                 JOINING_DATE DATETIME,
                 DEPARTMENT CHAR(25))""")

In [24]:
#Insert into Workers table

try:
    cnx = connection.connect(
        host     = "localhost", 
        database = 'ORG',
        user     = "root", 
        passwd   = mysql_pwd,
        use_pure = True)
    
    mycursor = cnx.cursor()

    sql = "INSERT INTO worker (WORKER_ID, FIRST_NAME, LAST_NAME, SALARY, JOINING_DATE, DEPARTMENT) VALUES (%s, %s, %s, %s, %s, %s)"
    val = [
        (1, "Monica", "Arora", 100000, "14-02-20 09.00.00", "HR"),
        (2, 'Niharika', 'Verma', 80000, '14-06-11 09.00.00', 'Admin'),
        (3, 'Vishal', 'Singhal', 300000, '14-02-20 09.00.00', 'HR'),
        (4, 'Amitabh', 'Singh', 500000, '14-02-20 09.00.00', 'Admin'),
        (5, 'Vivek', 'Bhati', 500000, '14-06-11 09.00.00', 'Admin'),
        (6, 'Vipul', 'Diwan', 200000, '14-06-11 09.00.00', 'Account'),
        (7, 'Satish', 'Kumar', 75000, '14-01-20 09.00.00', 'Account'),
        (8, 'Geetika', 'Chauhan', 90000, '14-04-11 09.00.00', 'Admin')
]
    mycursor.executemany(sql, val)

    cnx.commit()

    print(mycursor.rowcount, "record inserted.")


except Exception as e:
    cnx.close()
    print(str(e))

8 record inserted.


In [25]:
# Select from Workers table

try:
    cnx = connection.connect(
        host     = "localhost", 
        database = 'ORG',
        user     = "root", 
        passwd   = mysql_pwd,
        use_pure = True)
    
    sql = "SELECT * FROM WORKER"
    df = pd.read_sql(sql, cnx)
   
    print(df)

except Exception as e:
        print(str(e))
        print(pd.DataFrame(), str(e))

   WORKER_ID FIRST_NAME LAST_NAME  SALARY        JOINING_DATE DEPARTMENT
0          1     Monica     Arora  100000 2014-02-20 09:00:00         HR
1          2   Niharika     Verma   80000 2014-06-11 09:00:00      Admin
2          3     Vishal   Singhal  300000 2014-02-20 09:00:00         HR
3          4    Amitabh     Singh  500000 2014-02-20 09:00:00      Admin
4          5      Vivek     Bhati  500000 2014-06-11 09:00:00      Admin
5          6      Vipul     Diwan  200000 2014-06-11 09:00:00    Account
6          7     Satish     Kumar   75000 2014-01-20 09:00:00    Account
7          8    Geetika   Chauhan   90000 2014-04-11 09:00:00      Admin


In [37]:
df

,WORKER_ID,FIRST_NAME,LAST_NAME,SALARY,JOINING_DATE,DEPARTMENT
0,1,Monica,Arora,100000,2014-02-20 09:00:00,HR
1,2,Niharika,Verma,80000,2014-06-11 09:00:00,Admin
2,3,Vishal,Singhal,300000,2014-02-20 09:00:00,HR
3,4,Amitabh,Singh,500000,2014-02-20 09:00:00,Admin
4,5,Vivek,Bhati,500000,2014-06-11 09:00:00,Admin
5,6,Vipul,Diwan,200000,2014-06-11 09:00:00,Account
6,7,Satish,Kumar,75000,2014-01-20 09:00:00,Account
7,8,Geetika,Chauhan,90000,2014-04-11 09:00:00,Admin


In [40]:
def connect_to_mysql(myuser=None, mypasswd=None):
    """
    # connects to hard-coded MySQL database
    # returns tuple (cnx, err_code)
    # where 
    #     cnx - connection object 
    #     err_code = 0 on success, something else on error
    """
    try:
        cnx = mysql.connector.connect(
            host="localhost", 
            database = 'ORG',
            user='root', 
            passwd=mysql_pwd,
            use_pure=True)
        return cnx, 0
    except Exception as e:
        print(str(e))
        return None, str(e)

In [43]:
cnx,err = connect_to_mysql(myuser="root", mypasswd=os.getenv("MYSQL_PWD"))

In [44]:
query = """CREATE TABLE Bonus 
                 (WORKER_REF_ID INT,
                 BONUS_AMOUNT INT(10),
                 BONUS_DATE DATETIME,
                 FOREIGN KEY (WORKER_REF_ID)
                 REFERENCES Worker(WORKER_ID)
                 ON DELETE CASCADE);"""
do_sql(cnx=cnx, sql=query)

KeyboardInterrupt: 

In [38]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=mysql_pwd,
  database="ORG"
)

cursor = mydb.cursor()
cursor.execute("SHOW TABLES")

for table_name in cursor:
   print(table_name)

('Worker',)
